<a href="https://colab.research.google.com/github/jongbokhi/master_thesis_2023/blob/main/Cleaning%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import glob

In [ ]:
df_fri = pd.read_csv("/content/drive/MyDrive/new_cic_ids_2018 data/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv")
df_thu = pd.read_csv("/content/drive/MyDrive/new_cic_ids_2018 data/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv")

<ipython-input-3-19ec65f6f89a>:1: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fri = pd.read_csv("/content/drive/MyDrive/new_cic_ids_2018 data/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv")


In [ ]:
df_dataset = pd.concat([df_fri, df_thu], axis=0, ignore_index=True)

In [ ]:
df_dataset

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,16/02/2018 08:27:23,112640768,3,0,0,0,0,0,...,0,0,0.0,0,0,56300000.0,138.592929,56300000,56300000,Benign
1,0,0,16/02/2018 08:30:12,112641773,3,0,0,0,0,0,...,0,0,0.0,0,0,56300000.0,263.750829,56300000,56300000,Benign
2,35605,6,16/02/2018 08:26:55,20784143,23,44,2416,1344,240,64,...,20,2624734,0.0,2624734,2624734,9058214.0,0.0,9058214,9058214,Benign
3,0,0,16/02/2018 08:33:01,112640836,3,0,0,0,0,0,...,0,0,0.0,0,0,56300000.0,82.024387,56300000,56300000,Benign
4,23,6,16/02/2018 08:27:59,20,1,1,0,0,0,0,...,20,0,0.0,0,0,0.0,0.0,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097145,50111,6,15/02/2018 09:04:42,22,3,0,31,0,31,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2097146,443,6,15/02/2018 09:03:55,54682783,5,1,123,46,46,0,...,20,158783.0,0.0,158783,158783,54523813.0,0.0,54523813,54523813,Benign
2097147,443,6,15/02/2018 09:03:56,53682093,5,1,123,46,46,0,...,20,259719.0,0.0,259719,259719,53421756.0,0.0,53421756,53421756,Benign
2097148,443,6,15/02/2018 09:03:55,54683364,5,1,123,46,46,0,...,20,158870.0,0.0,158870,158870,54523593.0,0.0,54523593,54523593,Benign


In [ ]:
print('Class distribution: ')
df_dataset['Label'].value_counts()

Class distribution: 


Benign                      1442849
DoS attacks-Hulk             461912
DoS attacks-SlowHTTPTest     139890
DoS attacks-GoldenEye         41508
DoS attacks-Slowloris         10990
Label                             1
Name: Label, dtype: int64

In [ ]:
drop_columns =[
    'Dst Port',
    'Timestamp',
    # Features Without Observed Variance
    "CWE Flag Count",
    "Fwd Byts/b Avg",
    "Fwd Pkts/b Avg",
    "Fwd Blk Rate Avg",
    "Bwd Byts/b Avg",
    "Bwd Pkts/b Avg",
    "Bwd Blk Rate Avg",
    # Duplicate Column
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Fwd URG Flags',
    'Bwd URG Flags',
    'Flow Byts/s',  # This field had np.inf values during training, as such was removed
    'Flow Pkts/s'  # This field had np.inf values during training, as such was removed
]


In [ ]:
def clean_dataset(dataset):

        print(dataset["Label"].value_counts())
        print(f"Shape: {dataset.shape}")

        #1.Removing row with erroneous 'Label' label
        dataset.drop(dataset.loc[dataset["Label"] == "Label"].index, inplace=True)

        #2.Creating "Malicious" label from the 'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk','DoS attacks-GoldenEye', 'DoS attacks-Slowloris' labels
        dataset.replace(to_replace=['DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk',
                               'DoS attacks-GoldenEye', 'DoS attacks-Slowloris'], value="Malicious", inplace=True)

        #3.Forcing them to be strings
        dataset = dataset.astype({"Protocol": str})
        dataset = pd.get_dummies(dataset, columns=['Protocol'], drop_first=True)

        #4.making Label column the last column again
        dataset.insert(len(dataset.columns)-1, 'Label', dataset.pop('Label'))

        #5.Dropping some columns that may be irrelevant

        dataset.drop(columns=drop_columns, inplace=True)

        #6.Dropping missing and duplicate values
        dataset.dropna(inplace=True)
        dataset.drop_duplicates(inplace=True)

        #7.Replacing Benign labels with zeroes and Malicious labels with ones
        dataset.replace(to_replace="Benign", value=0, inplace=True)
        dataset.replace(to_replace="Malicious", value=1, inplace=True)

        print('============COMPLETE CLEANING============')
        print()
        print(dataset["Label"].value_counts())
        print(f"Shape: {dataset.shape}")
        dataset['Label'].value_counts()

        #8.Saving cleaned Dataset
        dataset.to_csv("/content/drive/MyDrive/new_cic_ids_2018 data/cleaned_dos.csv", index=False)




In [ ]:
df = clean_dataset(df_dataset)

Benign                      1442849
DoS attacks-Hulk             461912
DoS attacks-SlowHTTPTest     139890
DoS attacks-GoldenEye         41508
DoS attacks-Slowloris         10990
Label                             1
Name: Label, dtype: int64
Shape: (2097150, 80)
============COMPLETE CLEANING============

Class distribution: 


In [ ]:
df_dos = pd.read_csv("/content/drive/MyDrive/new_cic_ids_2018 data/cleaned_dos.csv")

In [ ]:
## procedure
#1. Split dataset into Training and Test (preserves the same proportions of examples in each class as observed in the original dataset): Stratified Train-Test Splits
#2. Featureselection using only Training dataset: Randomforest/XGboost
#3. Stratified K-fold Cross valisation for Training Dataset: applying Undersampling at the same time.
#4. Undersampling for Training dataset : Neighborhood Cleaning Rule

In [ ]:
"""
### Imbalanced dataset 절차

#1. Split dataset into Trianing and Test : Stratified Train-Test Splits
#2. Feature Selcetion : Feature Selcection은 Undersampling or Oversampling 전에 Training dataset만을 가지고 진행
#2.K-folds cross validation for Training dataset(Don`t use the test dataset) : Cross Validation은 Training dataset을 가지고 한다!
주의사항: dersampling or Oversanpling을 먼저 진행해서는 안된다. 먼저 진행하게 되면 validation과 testdatset의 정보를 사용하는 것과 마찬가지 인다. 절대 validation set과 test set의 정보는 evaluation전에 사용되서는 안된다.
1)각 폴더마다 Undersamplin or Oversampling을 진행
2)비교 모델들에 대해 각 K개의 training datasets으로 fitting 한후 k개의 validation set으로 평가한 후
3)각 모델의 평균 accuracy 비교후 최종 모델 선택

#3.최종 모델을 가지고 Training dataset을 이용해 Retrain 한다

#4. Test set을 가지고 Retrain된 최종 모델을 평가.

참고 site:
https://machinelearningmastery.com/combine-oversampling-and-undersampling-for-imbalanced-classification/
https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html
https://www.marcoaltini.com/blog/dealing-with-imbalanced-data-undersampling-oversampling-and-proper-cross-validation
"""